# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to: ***create functions***
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


## SQL Questions:


id, name, ratings, price
-id, reviews, will have own table

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. ***skip*** 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


In [59]:
import requests
import json
import mysql.connector
import config
import time

In [4]:
from mysql.connector import errorcode

url = https://api.yelp.com/v3/businesses/search
params = {term='pizza',
         location='East Village'
         }

## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [5]:
## Connect to DB server on AWS
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'yelp'
)

In [6]:
cur = cnx.cursor()

In [18]:
## Create new DB 

db_name = 'yelp'
def create_database(cur, database):
    try:
        cur.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cur.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cur, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database yelp does not exists.
Database yelp created successfully.


In [26]:
# Create a table for the Businesses
tables = {} #use dictionary to save diff create statements for each table
tables['businesses'] = (
    "CREATE TABLE businesses ("
    "  id varchar(50) NOT NULL," 
    "  name varchar(50) NOT NULL,"
    "  ratings float(32),"
    "  price varchar(16) NOT NULL,"
    "  PRIMARY KEY (id)"
    ") ENGINE=InnoDB")


tables['reviews'] = (
    "CREATE TABLE businesses ("
    "  id varchar(50) NOT NULL," 
    "  text varchar(50000) NOT NULL,"
    "  PRIMARY KEY (id)"
    ") ENGINE=InnoDB")



In [28]:
for table_name in tables:
    table_description = tables[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cur.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
               print("already exists.")
        else:
               print(err.msg)
    else:
           print("OK")

Creating table businesses: OK
Creating table reviews: already exists.


In [ ]:
# Create a table for the reviews

In [21]:
stmt = """ALTER TABLE businesses ADD review_count INT(10)"""
cur.execute(stmt)

## Part 2: Create ETL pipeline for the business data from the API

In [35]:
url = "https://api.yelp.com/v3/businesses/search"
api_key = '1PF8h8DMCwai1RQ0gaz0PGs-DKjbIrQ26j2cwASuiEUEC_GzWUzeK6Gv0FY82G7kltSUYIoKpa8j5rv9oaDeDA7u1kInZYP5lW8RwZq6zWu9RF9S9q97d0lJOtmxXXYx'
header = {'Authorization': 'Bearer %s' % api_key}
params = {'term': 'pizza',
         'location': 'East Village',
         'limit': '50'}

In [ ]:
def request 

In [14]:
req.status_code

200

In [25]:
json_ = json.loads(req.content)
json_['businesses']

{'id': '0UdC0VYX6aAvLiOcpR4MVQ',
 'alias': 'joes-pizza-new-york-7',
 'name': "Joe's Pizza",
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/9E2Um0ddicKyoWS-RGIXyg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/joes-pizza-new-york-7?adjust_creative=LZ8r20yLhIc3wfwCeX6gZw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LZ8r20yLhIc3wfwCeX6gZw',
 'review_count': 834,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.73331, 'longitude': -73.98763},
 'transactions': ['delivery', 'pickup'],
 'price': '$',
 'location': {'address1': '150 E 14th St',
  'address2': '',
  'address3': '',
  'city': 'New York',
  'zip_code': '10003',
  'country': 'US',
  'state': 'NY',
  'display_address': ['150 E 14th St', 'New York, NY 10003']},
 'phone': '+12123889474',
 'display_phone': '(212) 388-9474',
 'distance': 865.5899869147826}

In [96]:
all_results = []
for item in results:
    for thing in item['businesses']:
        all_results.append(thing)

In [94]:
# Write a function to make a call to the yelp API

def parser(data):
        if 'price' not in item.keys():
            bus = {
            'id' : item['id'],
            'name': item['name'],
            'rating': item['rating'],
            'price': 0,
            'review_count': item['review_count']}
        else:
            bus = {
            'id' : item['id'],
            'name': item['name'],
            'rating': item['rating'],
            'price': item['price'],
            'review_count': item['review_count']}
        
        return bus

In [99]:
to_sql = []
for item in all_results:
    to_sql.append(parser(item))

In [100]:
len(to_sql)

1000

In [75]:
def requests_from_yelp(url, header, params):
    req = requests.get(url, params=params, headers=header)
    data = json.loads(req.content)
    return data

def all_results(url, header, params):
    num = requests_from_yelp(url, params=params, header=header)['total']
    print('{} total matches found'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        params['offset'] = cur
        results.append(requests_from_yelp(url, params=params, header=header))
        time.sleep(1)
        cur += 50
    return results

In [76]:
results = all_results(url=url, header=header, params=params)

1400 total matches found


In [79]:
results[1]

{'businesses': [{'id': 'pGr5QuQU7Ur5GHLvXbT8bw',
   'alias': 'paulie-gees-brooklyn',
   'name': "Paulie Gee's",
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/1j39J4NABK_RBtO-l90QNg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/paulie-gees-brooklyn?adjust_creative=LZ8r20yLhIc3wfwCeX6gZw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LZ8r20yLhIc3wfwCeX6gZw',
   'review_count': 1205,
   'categories': [{'alias': 'pizza', 'title': 'Pizza'},
    {'alias': 'vegan', 'title': 'Vegan'},
    {'alias': 'italian', 'title': 'Italian'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.729546, 'longitude': -73.958568},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '60 Greenpoint Ave',
    'address2': '',
    'address3': '',
    'city': 'Brooklyn',
    'zip_code': '11222',
    'country': 'US',
    'state': 'NY',
    'display_address': ['60 Greenpoint Ave', 'Brooklyn, NY 11222']},
   'phone': '+13479873747',
   'display_ph

In [86]:
all_results = []
for item in results:
    for thing in item['businesses']:
        all_results.append(thing)

In [87]:
all_results[2]

{'id': 'nUpA3sY5b6dynC7aj0eV6A',
 'alias': 'rizzos-fine-pizza-new-york-4',
 'name': "Rizzo's Fine Pizza",
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/dnLj-NVZx4TUp608hDVBvw/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/rizzos-fine-pizza-new-york-4?adjust_creative=LZ8r20yLhIc3wfwCeX6gZw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LZ8r20yLhIc3wfwCeX6gZw',
 'review_count': 712,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'wine_bars', 'title': 'Wine Bars'},
  {'alias': 'beerbar', 'title': 'Beer Bar'}],
 'rating': 4.5,
 'coordinates': {'latitude': 40.720852, 'longitude': -73.984456},
 'transactions': ['delivery', 'pickup', 'restaurant_reservation'],
 'price': '$$',
 'location': {'address1': '17 Clinton St',
  'address2': '',
  'address3': '',
  'city': 'New York',
  'zip_code': '10002',
  'country': 'US',
  'state': 'NY',
  'display_address': ['17 Clinton St', 'New York, NY 10002']},
 'phone': '+16464541262',
 'disp

In [ ]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

In [92]:
def insert_restaurant(my_dict):
    stmt = """INSERT INTO businesses
    (id, name, ratings, price, review_count)
    VALUES (%(id)s,%(name)s,%(rating)s,%(price)s,%(review_count)s)"""
    cur.execute(stmt, my_dict)
    

def load_sql(list_dicts):
    first = """INSERT INTO businesses (id, name, ratings, price, review_count) VALUES"""
    str_list = []
    for item in list_dicts:
        second = items.values()
        second = [str[x] + ', ' for x in second]
        second = ''.join(second)[:-2]
        second = '('+ second +')'
        str_list.append(first+second)
    return str_list

In [106]:
for item in to_sql[63:]:
    insert_restaurant(item)
    cnx.commit()

IntegrityError: 1062 (23000): Duplicate entry 'Jsl8X_QjMLST0IyzTH4z5Q' for key 'PRIMARY'

In [1]:
# Write a function to take your parsed data and insert it into the DB

In [ ]:
# Write a script that combines the three functions above into a single process.

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [ ]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
def get_results(params):
#     consumer_key = "YOUR_KEY"
#     consumer_secret = "YOUR_SECRET"
#     token = "YOUR_TOKEN"
#     token_secret = "YOUR_TOKEN_SECRET"
   
    session = rauth.OAuth1Session(
    consumer_key = consumer_key
    ,consumer_secret = consumer_secret
    ,access_token = token
    ,access_token_secret = token_secret)
     
    request = session.get("http://api.yelp.com/v2/search",params=params)
   
  #Transforms the JSON API response into a Python dictionary
    data = request.json()
    session.close()
   
  return data

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews


In [ ]:
# Write a function to parse out the relevant information from the reviews

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
# combine the functions above into a single script  

## Part 4: Write SQL queries that will answer the questions posed. 

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [ ]:
def parse_results(results):
    
    return parsed_results

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time



def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0 #keeps track of where we are currently
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        db_insert(parsed)
        time.sleep(1) #Wait a second
        cur += 50

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```

### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)